In [1]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

In [2]:
from datetime import datetime as dtime

In [3]:
def rotaciona_corrente(zonal, meridional, phi):
    
    phi = np.deg2rad(phi)
    ul = zonal* np.cos(phi) + meridional * np.sin(phi)
    vl = -zonal * np.sin(phi) + meridional * np.cos(phi)
    
    return ul, vl

In [4]:
cd '/home/gus/Documentos/mestrado/dados-mestrado/dados-hycom'

/home/gus/Documentos/mestrado/dados-mestrado/dados-hycom


In [5]:
lista1 = sorted(glob('csv/*zonal.csv'))

In [6]:
lista2 = sorted(glob('csv/*meridional.csv'))

## carregando os dados

In [7]:
dfu = pd.read_csv(lista1[0], skip_blank_lines=True,\
                  error_bad_lines=False, delimiter= ';')

dfv = pd.read_csv(lista2[0], skip_blank_lines=True,\
                  error_bad_lines=False, delimiter=';')

lista_dfu = [dfu]
lista_dfv = [dfv]

In [8]:

for file1, file2 in zip(lista1[1:], lista2[1:]):

    data1 = pd.read_csv(file1, skip_blank_lines=True,
        error_bad_lines=False, delimiter=";")

    data2 = pd.read_csv(file2, skip_blank_lines=True,
        error_bad_lines=False, delimiter=";")

    lista_dfu.append(data1)
    lista_dfv.append(data2)


In [9]:
dfu.head()

,8m,16m,32m,40m,time
0,-0.357,-0.325,-0.311,-0.306,2016-12-02 12:00:00
1,-0.407,-0.336,-0.302,-0.300,2016-12-02 15:00:00
2,-0.382,-0.285,-0.228,-0.227,2016-12-02 18:00:00
3,-0.352,-0.291,-0.152,-0.150,2016-12-02 21:00:00
4,-0.308,-0.285,-0.225,-0.182,2016-12-03 00:00:00


In [10]:
dfu.tail()

,8m,16m,32m,40m,time
7,-0.340,-0.327,-0.309,-0.299,2016-12-03 09:00:00
8,-0.366,-0.326,-0.297,-0.288,2016-12-03 12:00:00
9,-0.389,-0.323,-0.275,-0.265,2016-12-03 15:00:00
10,-0.419,-0.322,-0.252,-0.240,2016-12-03 18:00:00
11,-0.431,-0.381,-0.241,-0.218,2016-12-03 21:00:00


## concatenando

In [11]:
dfu = pd.concat(lista_dfu, ignore_index=True)
dfv = pd.concat(lista_dfv, ignore_index=True)

In [12]:
dfu = dfu.iloc[:, 0:5]
dfv = dfv.iloc[:, 0:5]

In [13]:
dfu.shape

(1780, 5)

In [14]:
del data1, data2, lista_dfu, lista_dfv, lista1, lista2

In [15]:
time = np.copy(dfu.time)

In [16]:
time = [dtime.strptime(ii, "%Y-%m-%d %H:%M:%S") for ii in time]
time = np.array(time)

In [17]:
dfu.drop(['time'], inplace=True, axis=1)
dfv.drop(['time'], inplace=True, axis=1)

In [18]:
time.min(), time.max()

(datetime.datetime(2016, 12, 2, 12, 0), datetime.datetime(2017, 9, 29, 21, 0))

In [19]:
dfu = dfu *100;
dfv = dfv *100;

In [20]:
ul, vl = rotaciona_corrente(dfu, dfv, 60)

# realocando na matriz com intervalo de 1h

In [21]:
#plt.plot(time,'.r')

In [22]:
#min(time), max(time)

In [23]:
"""
tempoAjustado = pd.date_range(start='2016-12-01 12:00',\
                              end='2017-12-31 09:00', freq='1H')

ulAjustado = np.zeros([len(tempoAjustado), 4])
vlAjustado = np.zeros([len(tempoAjustado), 4])

ulAjustado[:] = np.NaN
vlAjustado[:] =  np.NaN

for line in range(len(time)):

    index = np.nonzero(tempoAjustado == time[line])

    ulAjustado[index[0], :] = ul.iloc[line, :]
    vlAjustado[index[0], :] = vl.iloc[line, :]
"""

"\ntempoAjustado = pd.date_range(start='2016-12-01 12:00',                              end='2017-12-31 09:00', freq='1H')\n\nulAjustado = np.zeros([len(tempoAjustado), 4])\nvlAjustado = np.zeros([len(tempoAjustado), 4])\n\nulAjustado[:] = np.NaN\nvlAjustado[:] =  np.NaN\n\nfor line in range(len(time)):\n\n    index = np.nonzero(tempoAjustado == time[line])\n\n    ulAjustado[index[0], :] = ul.iloc[line, :]\n    vlAjustado[index[0], :] = vl.iloc[line, :]\n"

In [24]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [25]:
"""
plt.plot(tempoAjustado, ulAjustado[:, 2],'.')
plt.ylim([-50,50])

"""

"\nplt.plot(tempoAjustado, ulAjustado[:, 2],'.')\nplt.ylim([-50,50])\n\n"

In [26]:
"""
limInferior = dtime.strptime('2016-11-18 20:00:00','%Y-%m-%d %H:%M:%S')
limSuperior = dtime.strptime('2017-09-19 08:00:00','%Y-%m-%d %H:%M:%S')
"""

"\nlimInferior = dtime.strptime('2016-11-18 20:00:00','%Y-%m-%d %H:%M:%S')\nlimSuperior = dtime.strptime('2017-09-19 08:00:00','%Y-%m-%d %H:%M:%S')\n"

In [27]:
"""
condtime = ((limInferior <= tempoAjustado) & (tempoAjustado<= limSuperior))
"""

'\ncondtime = ((limInferior <= tempoAjustado) & (tempoAjustado<= limSuperior))\n'

In [28]:
"""
dfu_cortado = pd.DataFrame(ulAjustado[condtime])
dfv_cortado = pd.DataFrame(vlAjustado[condtime])
"""

'\ndfu_cortado = pd.DataFrame(ulAjustado[condtime])\ndfv_cortado = pd.DataFrame(vlAjustado[condtime])\n'

In [29]:
"""
dfu_cortado.columns = dfu.columns
dfv_cortado.columns = dfv.columns
"""

'\ndfu_cortado.columns = dfu.columns\ndfv_cortado.columns = dfv.columns\n'

In [30]:
"""
dfu_cortado['time'] = tempoAjustado[condtime]
dfv_cortado['time'] = tempoAjustado[condtime]
"""

"\ndfu_cortado['time'] = tempoAjustado[condtime]\ndfv_cortado['time'] = tempoAjustado[condtime]\n"

In [31]:
ls

coordenadas.txt      hycom_ul_total_ajustado1h.csv  mvalidation.py
csv/                 hycom_vl_total_ajustado1h.csv  outputs/
hycom_nc4_to_csv.py  join_csv.py


## salvando os dados

In [32]:
"""
dfu_cortado.to_csv('hycom_ul_total_ajustado1h.csv', sep= ';', index=False)
dfv_cortado.to_csv('hycom_vl_total_ajustado1h.csv', sep= ';', index=False)
"""

"\ndfu_cortado.to_csv('hycom_ul_total_ajustado1h.csv', sep= ';', index=False)\ndfv_cortado.to_csv('hycom_vl_total_ajustado1h.csv', sep= ';', index=False)\n"

In [33]:
who

dfu	 dfv	 dtime	 file1	 file2	 glob	 np	 pd	 plt	 
rotaciona_corrente	 time	 ul	 vl	 


In [34]:
len(ul), len(vl), len(time)

(1780, 1780, 1780)

# Realocando os dados em intervalos de 3h sem buracos no tempo

### Essa matriz será usada na filtragem na banda submareal

In [35]:
tempoAjustado = pd.date_range(start='2016-12-02 12:00',\
                              end='2017-09-29 21:00', freq='3H')

ulAjustado = np.zeros([len(tempoAjustado), 4])
vlAjustado = np.zeros([len(tempoAjustado), 4])

ulAjustado[:] = np.NaN
vlAjustado[:] =  np.NaN

for line in range(len(time)):

    index = np.nonzero(tempoAjustado == time[line])

    ulAjustado[index[0], :] = ul.iloc[line, :]
    vlAjustado[index[0], :] = vl.iloc[line, :]

In [36]:
plt.plot(tempoAjustado, ulAjustado[:, 2],'-')
plt.ylim([-50,50])

(-50.0, 50.0)

In [37]:
limInferior = dtime.strptime('2016-11-18 20:00:00','%Y-%m-%d %H:%M:%S')
limSuperior = dtime.strptime('2017-09-19 08:00:00','%Y-%m-%d %H:%M:%S')

In [38]:
condtime = ((limInferior <= tempoAjustado) & (tempoAjustado<= limSuperior))

In [39]:
dfu_cortado = pd.DataFrame(ulAjustado[condtime])
dfv_cortado = pd.DataFrame(vlAjustado[condtime])

In [40]:
dfu_cortado.columns = dfu.columns
dfv_cortado.columns = dfv.columns

### interpolacao dos dados

In [46]:
dfu_cortado.interpolate(method='linear', axis=1,limit=4 ,inplace=True)
dfv_cortado.interpolate(method='linear', axis=1,limit=4 ,inplace=True)

In [49]:
plt.plot(dfu_cortado.iloc[:,0],'.-r')

In [ ]:
dfu_cortado['time'] = tempoAjustado[condtime]
dfv_cortado['time'] = tempoAjustado[condtime]

In [51]:
plt.plot(dfv_cortado.iloc[:,1])

In [52]:
dfu_cortado.to_csv('hycom_ul_total_ajustado3h.csv', sep= ';', index=False)
dfv_cortado.to_csv('hycom_vl_total_ajustado3h.csv', sep= ';', index=False)